# pymsql
- mysql을 파이썬에서 사용할 수 있는 라이브러리

```shell
pip install pymysql
```

In [4]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [122]:
import pymysql
import yaml

In [123]:
DATA_PATH = "data/"

with open(f"{DATA_PATH}config.yml", 'r') as f:
    CFG = yaml.full_load(f)

# connect 함수
- MySQL 서버와 연결할 수 있는 함수
- 연결객체를 반환
- 주요 파라미터
    - user: 계정
    - passwd: 패스워드
    - host: 호스트정보(ip 또는 도메인)
    - port: 포트정보
    - db: 선택할 DB 이름

In [124]:
conn = pymysql.connect(**CFG, db = 'mydata')
conn # 연결객체

# cursor 메서드
- 연결객체로 부터 cursor 객체를 반환
- cursor 객체란?
    - SQL 구문 실행하기 위한 객체

In [125]:
cursor = conn.cursor()
cursor # 커서객체

## execute 메서드
- 커서객체의 메서드로 전달받은 SQL문을 실행

In [145]:
sql = "select * from tb_titanic;"
cursor.execute(sql)

1309

## fetchall 메서드
- 커서객체의 메서드로 execute 메서드 실행 후 쿼리 실행 결과를 모두 가져올 때 사용

In [131]:
result = cursor.fetchall()
result[:5]

((1,
  0,
  3,
  'Braund, Mr. Owen Harris',
  'male',
  22.0,
  1,
  0,
  'A/5 21171',
  7.25,
  None,
  'S'),
 (2,
  1,
  1,
  'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  'female',
  38.0,
  1,
  0,
  'PC 17599',
  71.2833,
  'C85',
  'C'),
 (3,
  1,
  3,
  'Heikkinen, Miss. Laina',
  'female',
  26.0,
  0,
  0,
  'STON/O2. 3101282',
  7.925,
  None,
  'S'),
 (4,
  1,
  1,
  'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
  'female',
  35.0,
  1,
  0,
  '113803',
  53.1,
  'C123',
  'S'),
 (5,
  0,
  3,
  'Allen, Mr. William Henry',
  'male',
  35.0,
  0,
  0,
  '373450',
  8.05,
  None,
  'S'))

In [132]:
cursor.fetchall() # execute 실행 결과를 한번 반환하고 더이상 반환하지 않는다.

()

- dict 형식으로 받아오기

In [142]:
from pymysql.cursors import DictCursor
cursor = conn.cursor(DictCursor)

In [146]:
cursor.execute(sql)

1309

In [147]:
result = cursor.fetchall()

In [148]:
result[:2]

[{'passengerid': 1,
  'survived': 0,
  'pclass': 3,
  'name': 'Braund, Mr. Owen Harris',
  'gender': 'male',
  'age': 22.0,
  'sibsp': 1,
  'parch': 0,
  'ticket': 'A/5 21171',
  'fare': 7.25,
  'cabin': None,
  'embarked': 'S'},
 {'passengerid': 2,
  'survived': 1,
  'pclass': 1,
  'name': 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  'gender': 'female',
  'age': 38.0,
  'sibsp': 1,
  'parch': 0,
  'ticket': 'PC 17599',
  'fare': 71.2833,
  'cabin': 'C85',
  'embarked': 'C'}]

In [149]:
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)
    break

{'passengerid': 1, 'survived': 0, 'pclass': 3, 'name': 'Braund, Mr. Owen Harris', 'gender': 'male', 'age': 22.0, 'sibsp': 1, 'parch': 0, 'ticket': 'A/5 21171', 'fare': 7.25, 'cabin': None, 'embarked': 'S'}


In [150]:
sql = "select * from tb_titanic;"

cursor.execute(sql)

1309

## fetchone 메서드
- 하나씩 가져오기

In [151]:
cursor.fetchone()

{'passengerid': 1,
 'survived': 0,
 'pclass': 3,
 'name': 'Braund, Mr. Owen Harris',
 'gender': 'male',
 'age': 22.0,
 'sibsp': 1,
 'parch': 0,
 'ticket': 'A/5 21171',
 'fare': 7.25,
 'cabin': None,
 'embarked': 'S'}

## 커서 닫아주기

In [152]:
cursor.close()

In [153]:
# cursor.execute(sql) # 커서가 닫혔기 때문에 에러발생!!

In [154]:
with conn.cursor(DictCursor) as cursor:
    sql = "select * from tb_titanic;"
    cursor.execute(sql)
    result = cursor.fetchall()

In [155]:
# cursor.execute(sql)

## 연결 닫아주기

In [156]:
conn.close()

# DB 선택 없이 접속

In [157]:
conn = pymysql.connect(**CFG) # autocommit=True
conn

- autocommit 확인

In [158]:
conn.autocommit_mode

False

## DB 선택하기

In [159]:
conn.select_db("mydata") # usr mydata;

In [160]:
cursor = conn.cursor(DictCursor)

In [161]:
import json
with open(f"{DATA_PATH}state.json", 'r') as f:
    data = json.load(f)
type(data)

list

In [162]:
len(data)

50

In [163]:
data[0] # 주, 인구수, 10만명당 살인률

['Alabama', 4779736, 5.7]

In [164]:
sql = """
    create table if not exists tb_murder_rate(
        id int unsigned auto_increment primary key,
        state varchar(50) unique not null,
        population int,
        rate float
    );
"""
cursor.execute(sql)

0

In [165]:
columns_list = ["state", "population", "rate"]

In [166]:
values = [ f"'{item}'" if type(item) is str else str(item) for item in data[0]]
",".join(values)

"'Alabama',4779736,5.7"

In [108]:
sql = f"""
    insert into tb_murder_rate({",".join(columns_list)})
    values({",".join(values)});
"""
print(sql)


    insert into tb_murder_rate(state,population,rate)
    values('Alabama',4779736,5.7);



In [167]:
cursor.execute(sql)

0

In [168]:
conn.rollback() # insert, delete, update 을 취소

## PlaceHolder
- 동적 SQL 문을 구성할 때 유용하게 사용할 수 있는 기능
- 동적으로 들어가 위치에 `%s` 를 이용해 SQL 만든다.

In [112]:
sql = f"""
    insert into tb_murder_rate({",".join(columns_list)})
    values(%s, %s, %s)
"""
print(sql)


    insert into tb_murder_rate(state,population,rate)
    values(%s, %s, %s)



In [113]:
cursor.execute(sql, data[0])

1

In [116]:
conn.rollback()

In [117]:
cursor.executemany(sql, data)

50

In [118]:
conn.commit() # 실제 데이터 베이스에 반영

- 조회할 때 사용해보기

In [119]:
lst = [100000, 5]
sql = "select * from tb_murder_rate where population > %s and rate > %s"
cursor.execute(sql, lst)

16

In [120]:
result = cursor.fetchall()
result

[{'id': 3, 'state': 'Alabama', 'population': 4779736, 'rate': 5.7},
 {'id': 4, 'state': 'Alaska', 'population': 710231, 'rate': 5.6},
 {'id': 6, 'state': 'Arkansas', 'population': 2915918, 'rate': 5.6},
 {'id': 10, 'state': 'Delaware', 'population': 897934, 'rate': 5.8},
 {'id': 11, 'state': 'Florida', 'population': 18801310, 'rate': 5.8},
 {'id': 12, 'state': 'Georgia', 'population': 9687653, 'rate': 5.7},
 {'id': 15, 'state': 'Illinois', 'population': 12830632, 'rate': 5.3},
 {'id': 20, 'state': 'Louisiana', 'population': 4533372, 'rate': 10.3},
 {'id': 22, 'state': 'Maryland', 'population': 5773552, 'rate': 6.1},
 {'id': 24, 'state': 'Michigan', 'population': 9883640, 'rate': 5.4},
 {'id': 26, 'state': 'Mississippi', 'population': 2967297, 'rate': 8.6},
 {'id': 27, 'state': 'Missouri', 'population': 5988927, 'rate': 6.6},
 {'id': 30, 'state': 'Nevada', 'population': 2700551, 'rate': 6.0},
 {'id': 35, 'state': 'North Carolina', 'population': 9535483, 'rate': 5.1},
 {'id': 42, 'state'

In [121]:
cursor.close()
conn.close()